In [1]:
!pip install ultralytics
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
import pandas as pd
from ultralytics import YOLO
from roboflow import Roboflow
from google.colab import drive
import yaml

# Define Utility Functions

In [3]:
def retrieve_rf_project_data(api_key, workspace, project, version):
    '''
    Creates a Roboflow project folder and retrieves Roboflow project data into it

    Input: API Key - string,
           Workspace - string,
           Project - string,
           Version - integer
    Output: None
    '''

    rf = Roboflow(api_key=api_key)
    project = rf.workspace(workspace).project(project)
    version = project.version(version)
    dataset = version.download("yolov8")

In [4]:
def fix_image_paths(project):
    '''
    Fixes train, test, and valid image paths in data.yaml file

    Input: Project - string
    Output: None
    '''
    path = f'/content/{project}/'
    # Open data.yaml file
    with open(path+'data.yaml', 'r') as f:
        # Load yaml data into a dictionary
        data = yaml.safe_load(f)
        # Fix paths
        data['test'] = '/content/'+project+'/test/images'
        data['train'] = '/content/'+project+'/train/images'
        data['val'] = '/content/'+project+'/valid/images'

    # Save updated data.yaml file
    with open(path+'data.yaml', 'w') as f:
        yaml.dump(data, f)


# Mount Google MyDrive

In [5]:
# Mount drive
drive.mount('/content/drive')

Mounted at /content/drive


# Retrieve log of YOLO training runs

In [6]:
# Define base path
base_path = '/content/drive/MyDrive/AIBootcampDU/Project3/Runs/'

# Import Runs log into a Pandas DataFrame

df = pd.read_csv(base_path + 'RunsLog.csv')

display(df.head())

,model,# images,train,train-background,valid,valid-background,test,test-background,optimizer,cache,...,patience,iou,task,single_cls,show_labels,folder,weights,Roboflow version,Augmentations,Notes
0,yolov8m,1487,1041,NaN,258,NaN,188,NaN,AdamW,False,...,100,0.7,detect,False,True,Medium_model_1,0,Project-3-2,NaN,>1 class
1,yolov8m,1487,1041,NaN,258,NaN,188,NaN,AdamW,False,...,100,0.7,detect,False,True,Mediaum_model_2,0,Project-3-2,NaN,Continue previous medium run for total 110 epo...
2,yolov8m,1487,1041,NaN,258,NaN,188,NaN,AdamW,False,...,100,0.7,detect,False,True,Medium_model_3,0,Project-3-2,NaN,Continue previous medium run for total 120 epo...
3,yolov8x,1487,1041,NaN,258,NaN,188,NaN,AdamW,False,...,100,0.7,detect,False,True,XL-1,0,Project-3-2,NaN,>1 class
4,yolov8x,1487,1041,NaN,258,NaN,188,NaN,AdamW,False,...,100,0.7,detect,False,True,XL-2,0,Project-3-2,NaN,Continue previous medium run for total 110 epo...


In [7]:
# Remove rows corresponding to runs without weights folder
runs_df = df[df['weights']==1].reset_index(drop=True)
display(runs_df.head())

,model,# images,train,train-background,valid,valid-background,test,test-background,optimizer,cache,...,patience,iou,task,single_cls,show_labels,folder,weights,Roboflow version,Augmentations,Notes
0,yolov8m,8145,7170,2070.0,602,169.0,423,126.0,AdamW,True,...,100,0.7,detect,True,True,Medium_8195im_300ep,1,Project-3-4,Crop: 0-20%\nRotation: -15-+15º\nBrightness: -...,>1 class
1,yolov8m,8145,7170,2070.0,602,169.0,423,126.0,AdamW,True,...,100,0.7,detect,True,True,Medium_8195im_600ep,1,Project-3-4,Crop: 0-20%\nRotation: -15-+15º\nBrightness: -...,Continue previous medium run for total 600 epo...
2,yolov8m,8145,7170,2070.0,602,169.0,423,126.0,AdamW,True,...,100,0.7,detect,True,True,Medium_8195im_900ep_abandoned,1,Project-3-4,Crop: 0-20%\nRotation: -15-+15º\nBrightness: -...,Continue previous medium run for total 900 epo...
3,yolov8x,8145,7170,2070.0,602,169.0,423,126.0,AdamW,True,...,100,0.7,detect,True,True,XL_8195im_300ep_abandoned,1,Project-3-4,Crop: 0-20%\nRotation: -15-+15º\nBrightness: -...,>1 class\nFinishd early after 100 epochs w/o p...
4,yolov10m,8145,7170,2070.0,602,169.0,423,126.0,AdamW,True,...,100,0.7,detect,True,True,V10M_8195im_300ep,1,Project-3-4,Crop: 0-20%\nRotation: -15-+15º\nBrightness: -...,>1 class


# Retrieve Roboflow train, test, and validation datasets
Since different training runs are associated with different Roboflow projects, find the number of distinct projects

In [8]:
projects = runs_df['Roboflow version'].unique()

print(projects)

['Project-3-4' 'Project-3-5']


In [9]:
# Assign common variables needed
rf_api_key = "oqqGOXGuLE6ryiSqjEWA"
rf_workspace = "ai-bootcamp-project-3"
rf_project = "project-3-wiurd"

# Loop through projects
for project in projects:
    # Split version numbers from project string
    version = int(project.split('-')[-1])

    # Retrieve data for the version
    retrieve_rf_project_data(rf_api_key, rf_workspace, rf_project, version)

    # Fix the paths to the image data in the 'data.yaml' file
    fix_image_paths(project)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.68, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Project-3-4 in yolov8:: 100%|██████████| 16402/16402 [00:01<00:00, 9926.50it/s] 


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.68, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Project-3-5 in yolov8:: 100%|██████████| 16402/16402 [00:01<00:00, 9888.04it/s] 


In [13]:
results_dict = {'Model': [],
                'Folder': [],
                'Precision': [],
                'Recall': [],
                'MAP50': [],
                'MAP50-95': []}

confusion_matrices = [[]]

for run in runs_df.index:

    # Prepare parameter to instantiate the YOLO model
    model_name = runs_df['model'][run]
    folder = runs_df['folder'][run]
    model_path = base_path + folder + '/weights/best.pt'

    # Instantiate YOLO model
    model = YOLO(model_path)

    # Retrieve metrics
    metrics = model.val()

    # Add results to dictionary
    results_dict['Model'].append(model_name)
    results_dict['Folder'].append(folder)
    results_dict['Precision'].append(metrics.box.p[0,])
    results_dict['Recall'].append(metrics.box.r[0,])
    results_dict['MAP50'].append(metrics.box.map50)
    results_dict['MAP50-95'].append(metrics.box.ap[0,])

    confusion_matrices.append(metrics.confusion_matrix.matrix)

Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8m summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /content/Project-3-4/valid/labels.cache... 602 images, 169 backgrounds, 0 corrupt: 100%|██████████| 602/602 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/Project-3-4/valid/images/din_302_jpg.rf.342c3f2ae788dadd8e151b8bef5217af.jpg: 1 duplicate labels removed
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1123, len(boxes) = 1126. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:04<00:00,  8.24it/s]


                   all        602       1126      0.728       0.61       0.65      0.429
                     -        433       1126      0.728       0.61       0.65      0.429
Speed: 0.0ms preprocess, 3.7ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val11
Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8m summary (fused): 218 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


...

                   all        602       1126      0.792      0.552      0.637      0.425
Speed: 0.0ms preprocess, 5.2ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/val19
Ultralytics YOLOv8.2.68 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10m summary (fused): 369 layers, 16,451,542 parameters, 0 gradients, 63.4 GFLOPs


val: Scanning /content/Project-3-5/valid/labels.cache... 602 images, 169 backgrounds, 0 corrupt: 100%|██████████| 602/602 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/Project-3-5/valid/images/din_302_jpg.rf.342c3f2ae788dadd8e151b8bef5217af.jpg: 1 duplicate labels removed
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1123, len(boxes) = 1126. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:04<00:00,  8.45it/s]


                   all        602       1126      0.768       0.57      0.639      0.424
Speed: 0.0ms preprocess, 5.1ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/val20


In [11]:
# print(f"Model: {results_dict['Model']}, Folder: {results_dict['Folder']}")
# print(f"Confusion Matrix: {results_dict['Confusion Matrix']}")
# print(f"Precision: {results_dict['Precision']}, Recall: {results_dict['Recall']}, MAP50: {results_dict['MAP50']}, MAP50-95: {results_dict['MAP50-95']}")

In [14]:
results_df = pd.DataFrame(results_dict)
display(results_df)

print(confusion_matrices)

,Model,Folder,Precision,Recall,MAP50,MAP50-95
0,yolov8m,Medium_8195im_300ep,0.727976,0.610124,0.650352,0.429412
1,yolov8m,Medium_8195im_600ep,0.754258,0.587922,0.650033,0.426259
2,yolov8m,Medium_8195im_900ep_abandoned,0.745188,0.578153,0.641539,0.430134
3,yolov8x,XL_8195im_300ep_abandoned,0.782985,0.538314,0.627686,0.411876
4,yolov10m,V10M_8195im_300ep,0.775479,0.563055,0.637978,0.428406
5,yolov10m,V10M_8195im_300ep_04IoU_abandoned,1.000000,0.000000,0.320722,0.022025
6,yolov10m,V10M_8195im_300ep_05IoU_1cls_abandoned,0.717581,0.561872,0.608826,0.377785
7,yolov10m,V10M_8195im_300ep_05IoU_1cls,0.783769,0.571936,0.636226,0.420036
8,yolov10m,V10M_8195im_600ep_05IoU_1cls,0.792094,0.551516,0.637381,0.424642
9,yolov10m,V10M_8195im_900ep_05IoU_1cls_abandoned,0.768419,0.570160,0.639074,0.424282


[[], array([[        680,           0,         138],
       [          0,           0,           0],
       [        446,           0,           0]]), array([[        671,           0,         118],
       [          0,           0,           0],
       [        455,           0,           0]]), array([[        632,           0,          93],
       [          0,           0,           0],
       [        494,           0,           0]]), array([[        620,           0,         109],
       [          0,           0,           0],
       [        506,           0,           0]]), array([[        652,           0,         128],
       [          0,           0,           0],
       [        474,           0,           0]]), array([[          0,           0,           0],
       [          7,         621,         125],
       [          0,         498,           0]]), array([[        615,         118],
       [        511,           0]]), array([[        663,         114],
       [    